In [ ]:
import os
os.chdir('E:/code/iup/github')
%load_ext autoreload

In [ ]:
import pandas as pd
import sys
import os
import time
import shutil
import numpy as np
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
import math
import data_util as du
import exact_util as eu
import util
from data_util import Time
from brainflow.board_shim import BoardShim, BrainFlowInputParams, LogLevels, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations, WindowFunctions, DetrendOperations

import nolds

import iceland_tool as it
from config import Config, get_cfg_defaults
import features
from features import  perm_entropy, mean_absolue_value,sample_entropy, sorted_mean_absolue_value, root_mean_square, calc_log2,multiscale_entropy
import scipy as sp
import wfdb
import file_util as fu

In [ ]:
def curve(signal,method, record_time, fs, window_size=60, shift= 1,mean_filter_size = 10):
    y = eu.get_curve(signal, window_size=window_size, shift = shift, method=method)#zero_crossing_1
    if mean_filter_size != 0:
        DataFilter.perform_rolling_filter(y, mean_filter_size*fs, AggOperations.MEAN.value)

    return y

def analysis(signal,signal_fill,method, true_onsets, true_offsets , symbols, symbol_idxes,record_time):
    global global_threshold
    window_size, shift,base_window_size,base_shift, mean_filter_size = cfg.DETECT.WINDOW_SIZE,cfg.DETECT.SHIFT, cfg.DETECT.BASE_WINDOW_SIZE, cfg.DETECT.BASE_WINDOW_SHIFT, cfg.DETECT.MEAN_FILTER_SIZE


   
    features.global_threshold = du.std(signal)#0.0155 
    global_threshold = features.global_threshold

    
    signal_ = signal_fill

    
   
    y_origin = curve(signal_,method, record_time, fs, window_size=window_size*fs, shift= fs*shift, mean_filter_size = mean_filter_size)

    y_remove_base = eu.remove_base(y_origin, base_window_size = base_window_size*fs, shift = fs*10)

    
    y_all = [y_origin, y_remove_base]
    ans=[[],[]]
    use_thr = False
    for idx,y in enumerate(y_all):
        onsets,offsets = eu.detect_edge(y, 2, w_on = 0.3,w_off = 0.25, fs = fs, window_size = window_size, use_thr = use_thr)

       
    return y_origin, y_remove_base, y_remove_base_2
        
        

In [ ]:
def get_intervals(signal,fs, thr = 30):
    minute_interval = []
    minutes = du.get_total_minute(len(signal), fs)
    
    split = 1
    while minutes/split > thr:
        split+=1
    for i in range(split):
        if i == 0:
            onset = 0.5
        else:
            onset = (minutes -2)/split*i
        if i == split-1:
            offset = minutes -0.5
        else:
            offset = minutes/split*(i+1)
        
        minute_interval.append([onset, offset])
    return minute_interval

In [ ]:
global_threshold = 0
plot_result = False
record_time = Time(0,0,0)




cfg = get_cfg_defaults()
cfg.PARAM.CHANNEL = [9,10]
fs = cfg.PARAM.FS
channel = cfg.PARAM.CHANNEL
filter = cfg.PARAM.FILTER
method =sample_entropy
cfg.DETECT.METHOD_NAME = method.__name__
method_name = cfg.DETECT.METHOD_NAME 
print(cfg)

In [ ]:
from contraction_time import con_loc

def main(cfg):
    is_add_dur =  cfg.PARAM.IS_ADD_DUR
    cnt = 0

    names = it.get_names()
    
    from config import get_iup_filename
    import file_util as fu
    filename = get_iup_filename(cfg)
    print(filename)


    refs = []

    sig_lens = []
    base_iups = []
    iups = []
    iups_2= []
    save_signals = []
    save_names = []
    save_intervals = []

    for name in names:#names[idx-1:idx]:
        print(name)
        true_onsets, true_offsets = it.get_contractions_mark(name, fs)
        true_ref = np.array(con_loc[name])*fs*60
    #         if len(true_onsets)  == 0:
    #             continue
        signal = it.get_signal(name, filter = filter, channels = channel, target_fs = fs, normalize = cfg.PARAM.NORMALIZE)   
        symbols, symbol_idxes =  get_symbol(name, fs)
     #   print('symbols',symbols, symbol_idxes)
        if is_add_dur:
            add_dur = int(60*fs)
        else:
            add_dur = 0
        record_time  = Time(0,0,0)

        total_minute =  du.get_total_minute(len(signal), fs=fs)
        intervals = get_intervals(signal, fs = fs)



        for minute_interval in intervals:
            s_onset,s_offset = int(minute_interval[0]*60*fs), int(minute_interval[1]*60*fs)
            record_time  = Time(0,0,0)
            record_time.add_second((s_onset - add_dur)//fs)
            if minute_interval[1] - minute_interval[0] < 4:
                continue
            signal_= signal[s_onset:s_offset]
            if is_add_dur:
                signal_fill = fill_edge(signal_, add_dur, is_noise)
            else:
                signal_fill = signal_
            tmp_onsets = []
            tmp_offsets = []
            tmp_ref = []
            for i in range(len(true_onsets)):
               # if true_onsets[i]+ 70*fs >s_onset and true_offsets[i] - 70*fs < s_offset:
                if true_onsets[i] >s_onset and true_offsets[i] < s_offset:
                    tmp_onsets.append(true_onsets[i])
                    tmp_offsets.append(true_offsets[i])
            for i in range(len(true_ref)):
                if true_ref[i] >s_onset and true_ref[i] < s_offset:
                    tmp_ref.append(true_ref[i])


            tmp_onsets = np.array(tmp_onsets) - s_onset + add_dur 
            tmp_offsets = np.array(tmp_offsets) - s_onset  + add_dur 
            tmp_ref = np.array(tmp_ref)-s_onset+add_dur
            mids =(tmp_offsets + tmp_onsets)//2
            refs.append(tmp_ref)
        #@    print('mark cnt:',len(tmp_onsets))

            #symbol#
            idx = 0
            tmp_symbols = []
            tmp_symbol_idxes = []
            for idx in range(len(symbols)):
                if symbol_idxes[idx] > s_onset and symbol_idxes[idx] < s_offset:
                    tmp_symbol_idxes.append(symbol_idxes[idx])
                    tmp_symbols.append(symbols[idx])
            tmp_symbol_idxes = np.array(tmp_symbol_idxes) - s_onset + add_dur

            ######

            base_iup, iup,iup_2 = analysis(signal_,signal_fill,method,  tmp_onsets, tmp_offsets , tmp_symbols, tmp_symbol_idxes,record_time)
        #    peak_an = get_ans(iup, base_iup)
            
            if plot_result:
                
                if len(con_loc[name]) > 2:
                    ref_ = tmp_ref
                    util.plot_signal(iup, record_time, fs)
                  #  plt.scatter(np.array(onsets), np.ones((len(onsets)))*np.min(y),color ='red',marker = 'o')
                  #  plt.scatter(np.array(offsets),np.ones((len(offsets)))*np.min(y),color ='green', marker = '*')
                    plt.scatter(np.int32(ref_), iup[np.int32(ref_)],s=50, c = 'red',marker = 'o')
                    if len(peak_an)> 0:
                        plt.scatter(peak_an, iup[peak_an],s=50, c = 'black',marker = 'x')
                    util.plot_mark(iup,  tmp_onsets, tmp_offsets)
                    plot_symbol_points(iup, tmp_symbols, tmp_symbol_idxes, fs)
                    plt.show()
            # y, y_zr, signal, record_time =
            sig_lens.append(len(signal_fill))
            base_iups.append(base_iup)
            iups.append(iup)
            iups_2.append(iup_2)
            save_signals.append(signal_fill)
            save_names.append(name)
            save_intervals.append(minute_interval)
           # print(mids, r_rms_ans, r_en_ans, r_rms_en_ans)
    fu.save((save_names,save_signals, refs,sig_lens,base_iups, iups, iups_2, save_intervals), filename)